In [1]:
import requests
import tqdm
from bs4 import BeautifulSoup
import google.generativeai as genai
import re
import numpy as np
import pandas as pd

In [2]:
GOOGLE_GEMINI_API_KEY = get_secure_env('GOOGLE_GEMINI_API_KEY')

Key not found. Enter value for `GOOGLE_GEMINI_API_KEY`! ········


In [3]:
genai.configure(api_key=GOOGLE_GEMINI_API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash")

In [5]:
def load_file_contents(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = file.read()
    return data
file_path = 'data/AMZN_latest_10K.html'  
html_content = load_file_contents(file_path)
display_xml(html_content)

In [6]:
def remove_html_tags(html):
    soup = BeautifulSoup(html, "html.parser")
    return soup.get_text()
plain_text = remove_html_tags(html_content)
display_xml(plain_text)

In [7]:
# Prompt to give to the LLM
prompt = """
Read the 10K report of Amazon and provide summaries on the following topics.Try not to leave any field empty. Format your response exactly like below:
    1. AI-related Risks: Summarize how Amazon views risks associated with AI. Include specific mentions of potential challenges highlighted in the report.
    2. AI-related Opportunities: Summarize how Amazon perceives opportunities presented by AI. Highlight any areas where AI is seen as beneficial for the company.
    3. Investment in AI: Please summarize any specific investments in AI made by Amazon. Write as a paragraph.
        - Types of Investments: summarizing the investment in AI planned by the firm (if any) and clearly expressing in billions of dollars the latest year of investment in AI.Include mentions of investments in AI technologies, partnerships with AI firms, funding for AI research and development, acquisitions of AI-related companies, or any other relevant financial commitments. Clearly state the amount of these investments in billions of dollars.
        Please indicate that and provide any context available about AI-related initiatives.

    Ensure that all information is directly supported by the contents of the 10-K report. Reference specific sections or pages of the report that informed your summaries to verify the accuracy of the information.

    Here is the document:

"""
prompt += plain_text
prompt

'\nRead the 10K report of Amazon and provide summaries on the following topics.Try not to leave any field empty. Format your response exactly like below:\n    1. AI-related Risks: Summarize how Amazon views risks associated with AI. Include specific mentions of potential challenges highlighted in the report.\n    2. AI-related Opportunities: Summarize how Amazon perceives opportunities presented by AI. Highlight any areas where AI is seen as beneficial for the company.\n    3. Investment in AI: Please summarize any specific investments in AI made by Amazon. Write as a paragraph.\n        - Types of Investments: summarizing the investment in AI planned by the firm (if any) and clearly expressing in billions of dollars the latest year of investment in AI.Include mentions of investments in AI technologies, partnerships with AI firms, funding for AI research and development, acquisitions of AI-related companies, or any other relevant financial commitments. Clearly state the amount of these

In [21]:
response_text = model.generate_content(prompt)
response_text = response_text.text

In [22]:
print(response_text)

**1. AI-related Risks:**

Amazon's 10-K report does not explicitly address AI-related risks as a separate category. However, several risk factors indirectly relate to potential challenges posed by AI.  The report highlights the intense competition in the tech industry (Item 1A, Risk Factors), noting that "new and enhanced technologies, including...practical applications of artificial intelligence and machine learning...continue to increase our competition."  Furthermore, the report mentions risks related to intellectual property (Item 1A, Risk Factors), stating that "our and our customers’ use of artificial intelligence may result in increased claims of infringement or other claims, including those based on unauthorized use of third-party technology or content."  The potential for reputational damage from public perception of social or ethical issues concerning AI is also acknowledged (Item 1A, Risk Factors).  Finally, the potential for data loss or security breaches (Item 1A, Risk Fac

In [23]:
# Extracting sections using regular expressions
risk_section = re.search(r'\*\*1\. AI-related Risks:\*\*(.*?)\*\*2\. AI-related Opportunities:\*\*', response_text, re.DOTALL)
opportunity_section = re.search(r'\*\*2\. AI-related Opportunities:\*\*(.*?)\*\*3\. Investment in AI:\*\*', response_text, re.DOTALL)
investment_section = re.search(r'\*\*3\. Investment in AI:\*\*(.*)', response_text, re.DOTALL)

# Extracting the text from each section
risk_text = risk_section.group(1).strip() if risk_section else None
opportunity_text = opportunity_section.group(1).strip() if opportunity_section else None
investment_text = investment_section.group(1).strip() if investment_section else None

# Extracting investment values 
investment_values = re.findall(r'(\d+\.\d+)\s+billion', investment_text) if investment_text else []
if len(investment_values) == 1:
        investment_value = float(investment_values[0])
elif len(investment_values) > 1:
    investment_value = sum(float(value) for value in investment_values)
else:
    investment_value = "Not found"

def clean_text(raw_text):
    cleaned_text = re.sub(r'\\n+', '\n', raw_text)  # Replace multiple newline escapes with a single newline
    cleaned_text = cleaned_text.replace("\\", "")  # Remove backslashes
    cleaned_text = cleaned_text.replace("*", "")  # Remove asterisks
    cleaned_text = cleaned_text.strip()  # Remove leading and trailing whitespace
    return cleaned_text

# Clean the extracted texts
risk_text = clean_text(risk_text) if risk_text else None
opportunity_text = clean_text(opportunity_text) if opportunity_text else None
investment_text = clean_text(investment_text) if investment_text else None

# Compile extracted information into a Series
extracted_info = pd.Series({
    'Risk': risk_text,
    'Opportunity': opportunity_text,
    'Investment': investment_text,
    'Investment Value (Billion USD)': investment_value
})

In [24]:
display(extracted_info)

Risk                              Amazon's 10-K report does not explicitly addre...
Opportunity                       The 10-K report implicitly acknowledges AI as ...
Investment                        The Amazon 10-K report does not provide a spec...
Investment Value (Billion USD)                                                  4.0
dtype: object

In [43]:
tickers = ['NVDA', 'GOOGL', 'MSFT', 'AMZN', 'META', 'TSLA', 'IBM', 'INTC', 'CRM', 'ORCL']

In [46]:
# Analyse 10 different tech companies 10-K reports
all_info = []
for tic in tqdm.tqdm(tickers, desc='Extracting information'):
    extracted_info = None  
    file_path = f'data/{tic}_latest_10K.html'
    file = load_file_contents(file_path)  
    plain_text_file = remove_html_tags(file) 

    prompt = f"""
    Read the 10K report of {tic} and provide summaries on the following topics.Try not to leave any field empty. Format your response exactly like below:
    1. AI-related Risks: Summarize how {tic} views risks associated with AI. Include specific mentions of potential challenges highlighted in the report.
    2. AI-related Opportunities: Summarize how {tic} perceives opportunities presented by AI. Highlight any areas where AI is seen as beneficial for the company.
    3. Investment in AI: Please summarize any specific investments in AI made by {tic}. Write as a paragraph.
        - Types of Investments: summarizing the investment in AI planned by the firm (if any) and clearly expressing in billions of dollars the latest year of investment in AI.Include mentions of investments in AI technologies, partnerships with AI firms, funding for AI research and development, acquisitions of AI-related companies, or any other relevant financial commitments.
        Clearly state the amount of these investments in billions of dollars. 
        Please indicate that and provide any context available about AI-related initiatives.

    Ensure that all information is directly supported by the contents of the 10-K report. Reference specific sections or pages of the report that informed your summaries to verify the accuracy of the information.

    Here is the document:
    """
    prompt += plain_text_file

    response_text = model.generate_content(prompt)  
    response_text = response_text.text

    risk_section = re.search(r'\*\*1\. AI-related Risks:\*\*(.*?)\*\*2\. AI-related Opportunities:\*\*', response_text, re.DOTALL)
    opportunity_section = re.search(r'\*\*2\. AI-related Opportunities:\*\*(.*?)\*\*3\. Investment in AI:\*\*', response_text, re.DOTALL)
    investment_section = re.search(r'\*\*3\. Investment in AI:\*\*(.*)', response_text, re.DOTALL)

    risk_text = risk_section.group(1).strip() if risk_section else "Not found"
    opportunity_text = opportunity_section.group(1).strip() if opportunity_section else "Not found"
    investment_text = investment_section.group(1).strip() if investment_section else "Not found"

    investment_values = re.findall(r'(\d+\.\d+)\s+billion', investment_text) if investment_text else []
    if len(investment_values) == 1:
        investment_value = float(investment_values[0])
    elif len(investment_values) > 1:
        investment_value = sum(float(value) for value in investment_values)
    else:
        investment_value = "Not found"

    risk_text = clean_text(risk_text)
    opportunity_text = clean_text(opportunity_text)
    investment_text = clean_text(investment_text)

    extracted_info = pd.Series({
        'Ticker': tic,
        'Risk': risk_text,
        'Opportunity': opportunity_text,
        'Investment': investment_text,
        'Investment Value (Billion USD)': investment_value
    })

    all_info.append(extracted_info)

Extracting information: 100%|██████████| 10/10 [01:41<00:00, 10.17s/it]


In [47]:
try:
    tickers_base = ['NVDA', 'GOOGL', 'MSFT', 'AMZN', 'META', 'TSLA', 'IBM', 'INTC', 'CRM', 'ORCL']
    df = pd.DataFrame(all_info, index=tickers_base)
    df.to_csv('10k_extracted_info.csv')
    display(df)
except:
    print(f"Error")

,Ticker,Risk,Opportunity,Investment,Investment Value (Billion USD)
NVDA,NVDA,NVIDIA acknowledges several AI-related risks. ...,NVIDIA views AI as a significant opportunity a...,The 10-K report doesn't explicitly state the a...,45.3
GOOGL,GOOGL,Not found,Not found,Not found,Not found
MSFT,MSFT,Microsoft acknowledges the risks associated wi...,Microsoft views AI as a transformative technol...,The 10-K report doesn't provide a specific dol...,18.8
AMZN,AMZN,Not found,Not found,Not found,Not found
META,META,Not found,Not found,Not found,Not found
TSLA,TSLA,Not found,Not found,Not found,Not found
IBM,IBM,Not found,Not found,Not found,Not found
INTC,INTC,Not found,Not found,Not found,Not found
CRM,CRM,Not found,Not found,Not found,Not found
ORCL,ORCL,Oracle acknowledges significant risks associat...,Oracle sees substantial opportunities in AI. T...,The 10-K report does not provide a specific br...,26.2


**Conclusions**
1. Does the model accurately summarise the companies' views of AI as expressed in their 10K report? 
- The model's summary provides a generally accurate overview of each company's perspective on AI as in their 10-K reports. However, it lacks the depth that a human could offer. A human might be able to interpret subtle contextual clues that indicate a company's strategic positioning or concerns about AI, which the model may overlook. Moreover, humans can establish connections between sections of the report, providing a more integrated understanding of how various factors influence a company's approach to AI. Therefore, while the model can deliver a coherent overview, it may miss critical insights that a human expert could identify.

2. Does the Investment Value (Billion USD) cell provide a reliable and comparable estimation of the AI investment of each company? 
- The Investment Value (Billion USD) is not a reliable measure of each company's AI investments. It may overlook crucial details due to limitations in capturing and interpreting information. Explicit amounts provided may not fully reflect a company's AI-related expenditures, and different companies may define "investment in AI" differently. While it can offer some insights, it should be viewed with caution as it may not encompass the complete picture of each company's commitment to AI.

3. Would GPT have done a better or worse job at it? 
- No.GPT-4's content window is simply too small to process the large text of a 10K report.

In [38]:
# Ranking companies as 'Bullish', 'Bearish', or 'Neutral' 
data = pd.read_csv('10k_extracted_info.csv')
prompt = "Based on the following 10-K report analyses for each company, please rank them as 'Bullish', 'Bearish', or 'Neutral' on AI and provide a brief explanation for each ranking:"
for index, row in data.iterrows():
    company_info = (
        f"- Company: {row['Ticker']}:\n"
        f"- Risk: {row['Risk']}\n"
        f"- Opportunity: {row['Opportunity']}\n"
        f"- Investment: {row['Investment']}\n"
        f"- Investment Value (Billion USD): {row['Investment Value (Billion USD)']}\n\n"
    )
    prompt += company_info
prompt += """Please provide your rankings and explanations in this format:
             - Ranking: 
             - Explanation: """
ranking_text = model.generate_content(prompt)  
ranking_text = ranking_text.text
with open('my_AI_sentiment_ranking.txt', 'w', encoding='utf-8') as file:
    file.write(ranking_text)

In [39]:
display(ranking_text)

"Here's a ranking of the companies based on your provided 10-K analyses, along with explanations:\n\n**- Company: NVDA:**\n\n             - **Ranking:** Neutral\n             - **Explanation:**  The provided analysis lacks any information regarding NVDA's AI opportunities, risks, or investments.  A neutral stance is assigned due to insufficient data.\n\n**- Company: GOOGL:**\n\n             - **Ranking:** Bullish\n             - **Explanation:** Google's 10-K excerpt highlights significant opportunities in AI across search, DeepMind's breakthroughs, and Google Cloud services.  While investment figures are unavailable, the qualitative information points to a strong bullish outlook on AI.\n\n**- Company: MSFT:**\n\n             - **Ranking:** Bullish\n             - **Explanation:** Microsoft acknowledges AI risks but presents a very strong case for AI opportunities, particularly with Copilot and Azure AI.  While precise investment figures are missing, the description of substantial inve

While many major companies express optimism about AI, this bullish sentiment may lead to inflated stock prices. Despite their expectations for growth driven by AI, these companies also acknowledge significant risks, such as regulatory challenges and market fluctuations, which could render their current valuations unsustainable. If AI adoption slows or faces setbacks, it could result in a substantial market downturn. Thus, strategically shorting key AI players could capitalize on this potential bubble.